In [ ]:
import argparse

from ddbm import dist_util, logger
from datasets import load_data
from ddbm.resample import create_named_schedule_sampler
from ddbm.script_util import (
    model_and_diffusion_defaults,
    create_model_and_diffusion,
    sample_defaults,
    args_to_dict,
    add_dict_to_argparser,
    get_workdir
)
from ddbm.train_util import TrainLoop

import torch.distributed as dist
from torchvision import datasets

from pathlib import Path

import wandb
import numpy as np

from glob import glob
import os
from datasets.augment import AugmentPipe

## Data

In [ ]:
import os
import numpy as np
import cv2
from tqdm import tqdm
import torch
from torch.utils.data import Dataset
from dataclasses import dataclass
from torchvision import transforms as tr

In [ ]:
def get_channel_statistics(dataset):
    """
    Function for obtaining channel statistics (mean and deviation) for a dataset
    """
    channel_sum = np.zeros(3)
    channel_sq_sum = np.zeros(3)

    num_pixels = 0

    for img in tqdm(dataset):
        img = np.array(img) / 255.0
        channel_sum += img.mean(axis=(0, 1))
        channel_sq_sum += (img**2).mean(axis=(0, 1))
        num_pixels += 1

    channel_mean = channel_sum / num_pixels
    channel_std = np.sqrt(channel_sq_sum/num_pixels - channel_mean**2)

    return channel_mean, channel_std


def get_transforms(mean, std, img_size=256):
    train_transform = tr.Compose([
        tr.ToPILImage(),
        tr.Resize(img_size),
        # tr.RandomHorizontalFlip(),
        tr.ToTensor(),
        tr.Normalize(mean, std)
    ])

    test_transform = tr.Compose([
        tr.ToPILImage(),
        tr.Resize(img_size),
        tr.ToTensor(),
        tr.Normalize(mean, std)
    ])
    
    def de_normalize(tensor, normalized=True):
        tmp = tensor.cpu() * torch.from_numpy(std).reshape(3,1,1) + torch.from_numpy(mean).reshape(3,1,1)
        return tmp.permute(1, 2, 0)
    
    return train_transform, test_transform, de_normalize


class ImagesDataset(Dataset):
    def __init__(self, root_dir, transform=None, mode=None):
        """
        mode: one of "a", "b", None
        """
        super().__init__()
        self.root_dir = root_dir
        self.image_paths = [os.path.join(root_dir, f) for f in os.listdir(root_dir)]
        self.transform = transform
        self.mode = mode

    def __getitem__(self, index):
        img_path = self.image_paths[index]
        image = cv2.imread(img_path)[:, :, ::-1]  # Convert BGR to RGB
        if self.mode == "a":
            image = image[:, :image.shape[1]//2, :]
        elif self.mode == "b":
            image = image[:, image.shape[1]//2:, :]
        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.image_paths)


class PairedDataset(Dataset):
    def __init__(self, root_dir, transform_a=None, transform_b=None, random_flip=None, random_crop=None):
        super().__init__()
        self.root_dir = root_dir
        self.image_paths = [os.path.join(root_dir, f) for f in os.listdir(root_dir)]
        self.transform_a = transform_a
        self.transform_b = transform_b

    def __getitem__(self, index):
        img_path = self.image_paths[index]
        image = cv2.imread(img_path)[:, :, ::-1]  # Convert BGR to RGB
        image_a = image[:, :image.shape[1]//2, :]
        image_b = image[:, image.shape[1]//2:, :]
        if self.transform_a:
            image_a = self.transform_a(image_a)
        if self.transform_b:
            image_b = self.transform_b(image_b)
        return image_a, image_b

    def __len__(self):
        return len(self.image_paths)


@dataclass
class ImagesDatasetsClass:
    train_a: PairedDataset
    train_b: PairedDataset
    val_a: PairedDataset
    val_b: PairedDataset

@dataclass
class PairedDatasetsClass:
    train: PairedDataset
    val: PairedDataset

In [ ]:
def create_argparser():
    defaults = dict(
        data_dir="",
        dataset='edges2handbags',
        schedule_sampler="uniform",
        lr=1e-4,
        weight_decay=0.0,
        lr_anneal_steps=0,
        global_batch_size=2048,
        batch_size=-1,
        microbatch=-1,  # -1 disables microbatches
        ema_rate="0.9999",  # comma-separated list of EMA values
        log_interval=50,
        test_interval=500,
        save_interval=10000,
        save_interval_for_preemption=50000,
        resume_checkpoint="",
        exp='',
        use_fp16=False,
        fp16_scale_growth=1e-3,
        debug=False,
        num_workers=2,
        use_augment=False
    )
    defaults.update(model_and_diffusion_defaults())
    parser = argparse.ArgumentParser()
    add_dict_to_argparser(parser, defaults)
    return parser


if __name__ == "__main__":
    args = create_argparser().parse_args()
    main(args)

In [ ]:
"""
Train a diffusion model on images.
"""


def main(args):

    workdir = get_workdir(args.exp)
    Path(workdir).mkdir(parents=True, exist_ok=True)
    
    dist_util.setup_dist()
    logger.configure(dir=workdir)
    if dist.get_rank() == 0:
        name = args.exp if args.resume_checkpoint == "" else args.exp + '_resume'
        wandb.init(project="bridge", group=args.exp,name=name, config=vars(args), mode='online' if not args.debug else 'disabled')
        logger.log("creating model and diffusion...")
    

    data_image_size = args.image_size
    

    if args.resume_checkpoint == "":
        model_ckpts = list(glob(f'{workdir}/*model*[0-9].*'))
        if len(model_ckpts) > 0:
            max_ckpt = max(model_ckpts, key=lambda x: int(x.split('model_')[-1].split('.')[0]))
            if os.path.exists(max_ckpt):
                args.resume_checkpoint = max_ckpt
                if dist.get_rank() == 0:
                    logger.log('Resuming from checkpoint: ', max_ckpt)


    model, diffusion = create_model_and_diffusion(
        **args_to_dict(args, model_and_diffusion_defaults().keys())
    )
    model.to(dist_util.dev())

    if dist.get_rank() == 0:
        wandb.watch(model, log='all')
    schedule_sampler = create_named_schedule_sampler(args.schedule_sampler, diffusion)

    
    if args.batch_size == -1:
        batch_size = args.global_batch_size // dist.get_world_size()
        if args.global_batch_size % dist.get_world_size() != 0:
            logger.log(
                f"warning, using smaller global_batch_size of {dist.get_world_size()*batch_size} instead of {args.global_batch_size}"
            )
    else:
        batch_size = args.batch_size
        
    if dist.get_rank() == 0:
        logger.log("creating data loader...")

    data, test_data = load_data(
        data_dir=args.data_dir,
        dataset=args.dataset,
        batch_size=batch_size,
        image_size=data_image_size,
        num_workers=args.num_workers,
    )
    
    if args.use_augment:
        augment = AugmentPipe(
                p=0.12,xflip=1e8, yflip=1, scale=1, rotate_frac=1, aniso=1, translate_frac=1
            )
    else:
        augment = None
        
    logger.log("training...")
    TrainLoop(
        model=model,
        diffusion=diffusion,
        train_data=data,
        test_data=test_data,
        batch_size=batch_size,
        microbatch=args.microbatch,
        lr=args.lr,
        ema_rate=args.ema_rate,
        log_interval=args.log_interval,
        test_interval=args.test_interval,
        save_interval=args.save_interval,
        save_interval_for_preemption=args.save_interval_for_preemption,
        resume_checkpoint=args.resume_checkpoint,
        workdir=workdir,
        use_fp16=args.use_fp16,
        fp16_scale_growth=args.fp16_scale_growth,
        schedule_sampler=schedule_sampler,
        weight_decay=args.weight_decay,
        lr_anneal_steps=args.lr_anneal_steps,
        augment_pipe=augment,
        **sample_defaults()
    ).run_loop()



